# Analyze Fluorescent Intensity Data

## Import Modules

In [1]:
# Import data handling and analysis packages
import os
import glob
import pandas as pd
from scipy import stats

# Import plotting packages
import iqplot
import bokeh.io
from bokeh.io import output_file, show
from bokeh.layouts import column, row
bokeh.io.output_notebook()

Loading BokehJS ...

## Import and parse raw data

In [3]:
# Add experiment date here to apply to dataframe
path = os.path.abspath('')+'/raw_data_csvs/'
full_df = pd.DataFrame()
list_ = []

# For loop to bring in files and concatenate them into a single dataframe


for file_ in glob.glob(path + "/*.csv"):
    df = pd.read_csv(file_)
    # Determine Image name from file name
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]
    # Split values in ROI label
    df['Fluor'], df['ROI'] = zip(*df['Label'].map(lambda x: x.split(':')))
    # Split values in Image name column
    (df['ExptDate'], df['Treatment'], df['Dose'], df['Stains'], df['Embryo'], 
        df['Somites'], df['Section']) = zip(*df['Image'].map(lambda x: x.split('_')))
    list_.append(df)

full_df = pd.concat(list_)
full_df.head()

,,Label,Area,Mean,Min,Max,IntDen,RawIntDen,Image,Fluor,ROI,ExptDate,Treatment,Dose,Stains,Embryo,Somites,Section
0,1,H2BRFP:background,23.652,5.978,0,18,141.396,2744.0,"20190930_nSMase2MO_0,8mM_Pax7,H2BRFP,BREGFP,DA...",H2BRFP,background,20190930,nSMase2MO,"0,8mM","Pax7,H2BRFP,BREGFP,DAPI",Emb2,9ss,sec2
1,2,H2BRFP:background,21.951,6.026,0,16,132.275,2567.0,"20190930_nSMase2MO_0,8mM_Pax7,H2BRFP,BREGFP,DA...",H2BRFP,background,20190930,nSMase2MO,"0,8mM","Pax7,H2BRFP,BREGFP,DAPI",Emb2,9ss,sec2
2,3,H2BRFP:background,54.105,6.105,0,17,330.301,6410.0,"20190930_nSMase2MO_0,8mM_Pax7,H2BRFP,BREGFP,DA...",H2BRFP,background,20190930,nSMase2MO,"0,8mM","Pax7,H2BRFP,BREGFP,DAPI",Emb2,9ss,sec2
3,4,H2BRFP:Cntl,4041.420,247.745,0,2348,1001243.460,19430676.0,"20190930_nSMase2MO_0,8mM_Pax7,H2BRFP,BREGFP,DA...",H2BRFP,Cntl,20190930,nSMase2MO,"0,8mM","Pax7,H2BRFP,BREGFP,DAPI",Emb2,9ss,sec2
4,5,H2BRFP:Expt,3949.647,243.279,0,2272,960865.020,18647070.0,"20190930_nSMase2MO_0,8mM_Pax7,H2BRFP,BREGFP,DA...",H2BRFP,Expt,20190930,nSMase2MO,"0,8mM","Pax7,H2BRFP,BREGFP,DAPI",Emb2,9ss,sec2


## Calculate Corrected Total Cellular Fluorescence (CTCF) for each ROI

This will determine the fluorescence in each ROI (expression level), corrected for size and background intensity in the image, and normalize signaling reporter intensity (BRE::EGFP) over ubiquitous marker (H2B-RFP) to account for electroporation efficiency

In [6]:
# Define channels
expt_channel = 'BREGFP'
cntl_channel = 'H2BRFP'
cntl_construct = 'Control MO'
expt_construct = 'nSMase2 MO'

# Get a list of treatments
treatment_list = full_df.Treatment.unique()
treatment_list = treatment_list.tolist()

# Mean background values and group by Treatment, Embryo, Fluor, ROI and Section
mean_sections = ((full_df.groupby(['Treatment', 'Embryo', 'Fluor', 'ROI', 'Section', 'ExptDate'])
                  ['Area', 'Mean', 'IntDen']).mean())

# Loop trough treatments, performing each analysis and exporting CSV file for each treatment
for i in treatment_list:
    # Slice dataframe to process only embryos with given treatment
    treatment = i
    df_treatment = pd.DataFrame(mean_sections.xs(treatment))

    # Determine CTCF values = ROI IntDen - (background mean * ROI area)
    # Calculate background (background mean * ROI area)
    background_corr_cntl = (df_treatment.xs('background', level='ROI')['Mean'] 
                    * df_treatment.xs('Cntl', level='ROI')['Area'])
    background_corr_expt = (df_treatment.xs('background', level='ROI')['Mean'] 
                    * df_treatment.xs('Expt', level='ROI')['Area'])

    # Slice out only Cntl or Expt values in IntDen
    intdens_cntl = df_treatment.xs('Cntl', level='ROI')['IntDen'] 
    intdens_expt = df_treatment.xs('Expt', level='ROI')['IntDen'] 

    # Subtract background from IntDens to determine CTCF and concatenate into single dataframe
    sub_cntl = pd.DataFrame(intdens_cntl - background_corr_cntl)
    sub_expt = pd.DataFrame(intdens_expt - background_corr_expt)
    full_ctcf = pd.concat([sub_cntl, sub_expt], keys = ['Cntl', 'Expt'])
    full_ctcf.columns = ['CTCF']

    # Pull out TCFLef and H2BRFP values
    ctcf_expt_channel = full_ctcf.xs(expt_channel, level='Fluor')['CTCF'] 
    ctcf_cntl_channel = full_ctcf.xs(cntl_channel, level='Fluor')['CTCF'] 

    # Normalize for electroporation efficiency by determining TCFLef/RFP
    electroporation_norm = pd.DataFrame(ctcf_expt_channel / ctcf_cntl_channel)
    electroporation_norm.columns = ['CTCF']
    electroporation_norm.index.names = ['Side', 'Embryo', 'Section', 'ExptDate']

    # Average sections grouped by embryos before generating Expt/Cntl ratio
    averaged_sections = electroporation_norm.groupby(['Side','Embryo', 'ExptDate']).mean()

    # Pull out Cntl and Expt CTCFs
    ctcf_cntl = averaged_sections.xs('Cntl', level='Side')['CTCF'] 
    ctcf_expt = averaged_sections.xs('Expt', level='Side')['CTCF'] 

    # Generate ratios as Expt/Cntl
    ratios_sections = pd.DataFrame(ctcf_expt / ctcf_cntl)
    ratios_sections.columns = ['Expt/Cntl CTCF']

    # Normalize individual values to mean of control group
    norm_cntl = ctcf_cntl/(float(ctcf_cntl.mean()))
    norm_cntl = pd.DataFrame(norm_cntl)
    norm_cntl.columns = [cntl_construct + ' normCTCF']
    norm_expt = ctcf_expt/(float(ctcf_cntl.mean()))
    norm_expt = pd.DataFrame(norm_expt)
    norm_expt.columns = [expt_construct + ' normCTCF']

    # Combine processed values into single dataframe and output as csv file 'Results.csv'
    ctcf_cntl = pd.DataFrame(ctcf_cntl)
    ctcf_cntl.columns = ['Cntl CTCF']
    ctcf_expt = pd.DataFrame(ctcf_expt)
    ctcf_expt.columns = ['Expt CTCF']
    results = (pd.concat([ctcf_cntl, ctcf_expt, ratios_sections, norm_cntl, norm_expt], axis=1, sort=True)).reset_index()
    results['ID'] = results.ExptDate.str.cat(results.Embryo)
    results['Readout'] = str(expt_channel)
    results.to_csv(expt_construct + ' ' + expt_channel + ' normCTCF Results.csv')
    
results

,Embryo,ExptDate,Cntl CTCF,Expt CTCF,Expt/Cntl CTCF,Control MO normCTCF,nSMase2 MO normCTCF,ID,Readout
0,Emb1,20190812,1.357554,0.612655,0.451293,1.119580,0.505258,20190812Emb1,BREGFP
1,Emb1,20190930,2.711450,2.428278,0.895564,2.236142,2.002609,20190930Emb1,BREGFP
2,Emb2,20190930,1.847029,1.503528,0.814026,1.523251,1.239965,20190930Emb2,BREGFP
3,Emb3,20190930,0.648103,0.481537,0.742995,0.534493,0.397125,20190930Emb3,BREGFP
4,Emb4,20190812,1.490497,0.810699,0.543912,1.229218,0.668586,20190812Emb4,BREGFP
5,Emb5,20190812,0.369388,0.171425,0.464080,0.304636,0.141375,20190812Emb5,BREGFP
6,Emb6,20190812,0.515897,0.284930,0.552299,0.425462,0.234982,20190812Emb6,BREGFP
7,Emb6,20190930,0.760539,0.297187,0.390758,0.627219,0.245091,20190930Emb6,BREGFP


## Plot and perform statistical analysis

In [7]:
data = results.reset_index()

# Build Stripbox plot
stripbox = iqplot.stripbox(
                    # Data to plot
                        data=data,
                        q='Expt/Cntl CTCF', q_axis='y',
                        cats=['Readout'], 

                    # Plot details
                        jitter=True, jitter_kwargs=dict(width=0.3),
                        marker_kwargs=dict(alpha=0.8, size=8
#                                            ,color='darkgray'
                                          ),
                        box_kwargs=dict(line_color='black', line_width=1.5),
                        whisker_kwargs=dict(line_color='black', line_width=1.5),
                        median_kwargs=dict(line_color='black', line_width=2),
                        top_level='box',
                        frame_width=150, frame_height=300,

                    # Plot customizations
#                         order=targets,
                        y_range=(0,1.5),
                        y_axis_label='Normalized CTCF (Expt/Cntl)',
                        x_axis_label=expt_construct,
                        show_legend=False,
)

# Final customizations
stripbox.axis.axis_label_text_font_size = '16px'
stripbox.axis.major_label_text_font_size = '16px'
stripbox.axis.axis_label_text_font_style = 'bold'
stripbox.xaxis.major_label_text_font_style = 'italic'

# View plot
show(stripbox)

In [9]:
################### Isolate data for analysis ###################
# Pull out only cells and treaments of interest, and rename ROIs with the appropriate constructs
data=results.reset_index()
data=data.loc[data['Readout'] == expt_channel]
data=data.filter(['ID', cntl_construct + ' normCTCF', expt_construct + ' normCTCF'])
data=data.melt(id_vars=['ID'], var_name='ROI', value_name='Norm CTCF')
data['ROI'] = data['ROI'].str.replace(' normCTCF', '')

################### Plot as strip plot ###################
# Plot as strip plot
p1 = iqplot.strip(data=data
                ,q='Norm CTCF', q_axis='y'
                ,cats=['ROI'], parcoord_column='ID'
#                 ,y_range=(0,2)
                ,frame_height = 300, frame_width = 150
                ,y_axis_label= 'Normalized CTCF'
#                 ,color_column='Image'
                ,marker_kwargs=dict(size=5,color='black')
                ,parcoord_kwargs=dict(line_width=1,color='gray')
#                 ,show_legend=True
              )

# p1.axis.axis_label_text_font_style = 'bold italic'
p1.axis.axis_label_text_font_size = '14px'
p1.axis.major_label_text_font_size = '12px'
p1.axis.axis_label_text_font_style = 'normal'
p1.xaxis.major_label_orientation = 7

show(row(p1))

################### Perform statistical analysis ###################

# Perform Paired t test 
cntl = data.loc[data['ROI'] == cntl_construct]['Norm CTCF']
expt = data.loc[data['ROI'] == expt_construct]['Norm CTCF']
ttest = stats.ttest_rel(cntl,expt)

# Display test results
print('Paired t-test results: \n\t\t statistic=' + str(ttest[0]) + 
    '\n\t\t p-value=' + str(ttest[1]))

Paired t-test results: 
		 statistic=4.982362472314212
		 p-value=0.0015967199733918304
